In [1]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model


In [2]:
# Load a pre-trained model (e.g., SSD MobileNet from TensorFlow Hub)
model = tf.keras.applications.MobileNetV2(weights='imagenet')


14536120/14536120 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [3]:
video_capture = cv2.VideoCapture('/content/video_yolov4.sh')


In [4]:
# Define output video writer
output_path = 'output_video.avi'
frame_width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(video_capture.get(cv2.CAP_PROP_FPS))
output_writer = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'XVID'), fps, (frame_width, frame_height))


In [5]:
# Process the video frame by frame
while video_capture.isOpened():
    ret, frame = video_capture.read()
    if not ret:
        break

    # Resize the frame to fit the model's input requirements
    input_frame = cv2.resize(frame, (224, 224))
    input_frame = np.expand_dims(input_frame, axis=0)
    input_frame = tf.keras.applications.mobilenet_v2.preprocess_input(input_frame)

    # Run inference
    predictions = model.predict(input_frame)

    # Decode and display predictions (example for classification)
    decoded_predictions = tf.keras.applications.mobilenet_v2.decode_predictions(predictions, top=3)[0]
    for i, (imagenet_id, label, score) in enumerate(decoded_predictions):
        cv2.putText(frame, f"{label}: {score:.2f}", (10, 30 + i * 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Write the processed frame to the output video
    output_writer.write(frame)

    # Display the frame
    cv2.imshow('Object Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

In [6]:
# Release resources
video_capture.release()
output_writer.release()
cv2.destroyAllWindows()